# Loading Deepmind's Diplomacy trajectory

In [ ]:
import numpy as np

In [ ]:
action_outputs = np.load(file='../data/actions_outputs.npz', allow_pickle=True)

In [ ]:
action_outputs[1][0]

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame.from_dict(action_outputs[0][0]).corr(method='pearson')

In [ ]:
legal_actions = np.load(file='../data/legal_actions.npz', allow_pickle=True)

In [ ]:
len(legal_actions)

In [ ]:
legal_actions[0].shape

In [ ]:
observations = np.load(file='../data/observations.npz', allow_pickle=True)

In [ ]:
len(observations)

In [ ]:
observations[1]

In [ ]:
step_outputs = np.load(file='../data/step_outputs.npz', allow_pickle=True)

In [ ]:
len(step_outputs)

In [ ]:
step_outputs[0]

# Loading human plays' data

In [124]:
import json
import pandas as pd
import mmap

In [125]:
# Load the json data
games_jsons = []
with open("../data/diplomacy-v1-27k-msgs/standard_no_press.jsonl", "r+b") as json_file:
    with mmap.mmap(json_file.fileno(), length=0, access=mmap.ACCESS_READ) as mmap_object:
        for i, line in enumerate(iter(mmap_object.readline, b"")):
            games_jsons.append(json.loads(line.decode("utf-8")))
            if i == 20:
                break

In [126]:
# Convert to a pandas dataframe
df = pd.DataFrame(games_jsons)

In [127]:
df = df[df["map"] == "standard"].reset_index(drop=True)

In [128]:
df

,id,map,rules,phases
0,uXFQ2zgI-DUrgwlS,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."
1,X9hvE_k6LbQrauYc,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."
2,lVp6PZxk3Jpufc9Z,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."
3,rW_Q9TpidGLGD2EL,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."
4,HYc16KDWi8zHNlmn,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."
5,_xZyPB0yRDQjRB4x,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."
6,CRbbNicSK5Jc-qmb,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."
7,ZSFmLzi-Th6lbpxy,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."
8,57xpw6Z-8IKo2hDK,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."
9,sdKZrT-i_BvEZsFU,standard,"[NO_PRESS, POWER_CHOICE]","[{'name': 'S1901M', 'state': {'timestamp': 154..."


In [129]:
games = df['phases']

In [130]:
games.apply(lambda x: x[0])

0     {'name': 'S1901M', 'state': {'timestamp': 1542...
1     {'name': 'S1901M', 'state': {'timestamp': 1542...
2     {'name': 'S1901M', 'state': {'timestamp': 1542...
3     {'name': 'S1901M', 'state': {'timestamp': 1542...
4     {'name': 'S1901M', 'state': {'timestamp': 1542...
5     {'name': 'S1901M', 'state': {'timestamp': 1542...
6     {'name': 'S1901M', 'state': {'timestamp': 1542...
7     {'name': 'S1901M', 'state': {'timestamp': 1542...
8     {'name': 'S1901M', 'state': {'timestamp': 1542...
9     {'name': 'S1901M', 'state': {'timestamp': 1542...
10    {'name': 'S1901M', 'state': {'timestamp': 1542...
11    {'name': 'S1901M', 'state': {'timestamp': 1542...
12    {'name': 'S1901M', 'state': {'timestamp': 1542...
13    {'name': 'S1901M', 'state': {'timestamp': 1542...
Name: phases, dtype: object

In [131]:
type(games)

pandas.core.series.Series

In [132]:
for game in games:
    for ix, iy in enumerate(game):
        game[ix]['phase_id'] = ix

In [133]:
def flatten_json(nested_json):
    out = {}
    x = nested_json
    
    out['game_id'] = x['state']['game_id']
    out['phase_id'] = x['phase_id']
    out['phase_name'] = x['name']
    results_units = [x for x in x['results']]
    # print("results are: ", results_units)
    assert x['orders'].items() 
    for key, value in x['orders'].items():
        if value == None:
            out['coordinator'] = key
            out['type'] = 'N'
            out['current_location'] = 'N'
            out['action'] = 'N'
            out['results'] = 'N'
            out['via'] = 'N'
            out['impact_location'] = 'N'
            yield(out)
        else:
            if value:
                for ix in value:
                    out['coordinator'] = key
                    out['type'] = ix.split()[0]
                    out['current_location'] = ix.split()[1]
                    out['action'] = ix.split()[2]
                    unit = ix.split()[0]+ ' ' + ix.split()[1]
                    # print("unit result is: ", unit , unit in results_units, results_units)
                    if x['results']:
                        out['results'] = x['results'][unit]
                    else:
                        print("empty results")
                        print(ix)
                    if unit in results_units:
                        results_units.remove(unit)
                    out['via'] = 'N'
                    out['impact_location'] = []
                    if out['action'] == '-' or out['action'] == 'R':
                        assert len(ix.split()) == 4 or len(ix.split()) == 5, ix
                        out['impact_location'] = ix.split()[3]
                        if ix.split()[-1] == 'VIA':
                            out['via'] = ix.split()[-1]
                yield(out)

    if len(results_units)>0:
        # print("adding new rows: ", results_units)
        for ix in results_units:
            out['coordinator'] = 'N'
            for key in x["state"]["units"]:
                if ix in x["state"]["units"][key]:
                    # print(key, value, "found")
                    out['coordinator'] = key
            out['type'] = ix.split()[0] if len(ix.split()) > 1 else 'N'
            out['current_location'] = ix.split()[1] if len(ix.split()) > 1 else ix
            out['action'] = 'N'
            out['results'] = x['results'][ix]
            out['via'] = 'N'
            out['impact_location'] = 'N'
            yield(out)

In [134]:
complete_df = pd.DataFrame()
for idx, game in enumerate(games):
  # print("game ", idx)
  for idx, phase in enumerate(game):
    # print("phase ", idx)
    row_i = flatten_json(phase)
    if row_i != None:
      for ix in row_i:  
        row_df = pd.DataFrame(ix.items()).T
        complete_df = pd.concat((complete_df,row_df), axis=0, ignore_index=True)

new_header = complete_df.iloc[0] #grab the first row for the header
complete_df = complete_df[1:] #take the data less the header row
complete_df.columns = new_header #set the header row as the df header
complete_df = complete_df.loc[::2]
complete_df = complete_df.reset_index(drop=True)

empty results
A VEN D


In [135]:
complete_df

,game_id,phase_id,phase_name,coordinator,type,current_location,action,results,via,impact_location
0,uXFQ2zgI-DUrgwlS,0,S1901M,AUSTRIA,A,BUD,-,[],N,SER
1,uXFQ2zgI-DUrgwlS,0,S1901M,ENGLAND,F,LON,-,[],N,NTH
2,uXFQ2zgI-DUrgwlS,0,S1901M,FRANCE,A,MAR,-,[],N,SPA
3,uXFQ2zgI-DUrgwlS,0,S1901M,GERMANY,F,KIE,H,[],N,[]
4,uXFQ2zgI-DUrgwlS,0,S1901M,ITALY,A,ROM,H,[],N,[]
...,...,...,...,...,...,...,...,...,...,...
2512,7uKSGh-EG86tfpyh,31,S1908R,ENGLAND,N,N,N,N,N,N
2513,7uKSGh-EG86tfpyh,31,S1908R,FRANCE,N,N,N,N,N,N
2514,7uKSGh-EG86tfpyh,31,S1908R,GERMANY,N,N,N,N,N,N
2515,7uKSGh-EG86tfpyh,31,S1908R,ITALY,N,N,N,N,N,N


In [136]:
complete_df['unique_unit_id'] = -1

In [137]:
complete_df

,game_id,phase_id,phase_name,coordinator,type,current_location,action,results,via,impact_location,unique_unit_id
0,uXFQ2zgI-DUrgwlS,0,S1901M,AUSTRIA,A,BUD,-,[],N,SER,-1
1,uXFQ2zgI-DUrgwlS,0,S1901M,ENGLAND,F,LON,-,[],N,NTH,-1
2,uXFQ2zgI-DUrgwlS,0,S1901M,FRANCE,A,MAR,-,[],N,SPA,-1
3,uXFQ2zgI-DUrgwlS,0,S1901M,GERMANY,F,KIE,H,[],N,[],-1
4,uXFQ2zgI-DUrgwlS,0,S1901M,ITALY,A,ROM,H,[],N,[],-1
...,...,...,...,...,...,...,...,...,...,...,...
2512,7uKSGh-EG86tfpyh,31,S1908R,ENGLAND,N,N,N,N,N,N,-1
2513,7uKSGh-EG86tfpyh,31,S1908R,FRANCE,N,N,N,N,N,N,-1
2514,7uKSGh-EG86tfpyh,31,S1908R,GERMANY,N,N,N,N,N,N,-1
2515,7uKSGh-EG86tfpyh,31,S1908R,ITALY,N,N,N,N,N,N,-1


# Temporal Binding

In [111]:
def assign_unit_id(phase_df, source_unit_id_map, dest_unit_id_map, _id):

    # fror each row in the phase df
    for idx, row in phase_df.itertuples():

        # if current location or type of army in invalid skip the row (we only deal with valid orders)
        if row.current_location == 'N' or row.type == 'N':
            continue
        
        # get the current location of the unit
        source_unit = row.type + ' ' + row.current_location

        # if the location is not in the map, add it to the map (in other phases the same unit can be used, hence checking the condition _ dictionaries are global, have data across phases)
        if source_unit not in source_unit_id_map:
            source_unit_id_map[source_unit] = _id
            _id += 1

        # destination dict is synced with source dict after the loop, so that we can use updated info at the beginning of each assignment
        phase_df.set_value(row.Index, row.unique_unit_id, source_unit_id_map[source_unit], 1)

        # print(idx, row)

        if row['action'] == '-':
            result = row.results
            if isinstance(result, list):
                if len(result) == 0:
                    dest_location = row.impact_location
                    try:
                        dest_unit = row.type + ' ' + dest_location
                    except:
                        print("dest location error", row)
                        return
                    if dest_unit not in dest_unit_id_map:
                        dest_unit_id_map[dest_unit] = source_unit_id_map[source_unit]
                        source_unit_id_map.pop(source_unit)
                elif 'disband' in result:
                    source_unit_id_map.pop(source_unit)
                    
        elif row.action == 'R':
            result = row.results
            if isinstance(result, list):
                if len(result) == 0:
                    dest_location = row.impact_location
                    dest_unit = row.type + ' ' + dest_location
                    if dest_unit not in dest_unit_id_map:
                        dest_unit_id_map[dest_unit] = source_unit_id_map[source_unit]
                        source_unit_id_map.pop(source_unit)
                elif 'disband' in result:
                    source_unit_id_map.pop(source_unit)

        elif row['action'] == 'D':
            source_unit_id_map.pop(source_unit)
        
        elif row['action'] == 'B':
            assert source_unit in source_unit_id_map

        # phase_df.loc[idx] = row

    # merge the source and destination dictionaries into one
    source_unit_id_map.update(dest_unit_id_map)
    # remove the destination dict (values get updated based on old data if we don't do this)
    dest_unit_id_map = {}

    return phase_df, source_unit_id_map, dest_unit_id_map, _id


In [117]:
unique_games = complete_df["game_id"].unique()

In [118]:
unique_games

array(['uXFQ2zgI-DUrgwlS', 'X9hvE_k6LbQrauYc', 'lVp6PZxk3Jpufc9Z',
       'rW_Q9TpidGLGD2EL', 'HYc16KDWi8zHNlmn', '_xZyPB0yRDQjRB4x',
       'CRbbNicSK5Jc-qmb', 'ZSFmLzi-Th6lbpxy', '57xpw6Z-8IKo2hDK',
       'sdKZrT-i_BvEZsFU', 'p6m8jMuDPsM0dtUh', 'oIIaa9M0NHem5XE_',
       '5F6upbHF5NZlWuY_', '7uKSGh-EG86tfpyh'], dtype=object)

In [119]:
pd.__version__

'1.3.5'

In [120]:
complete_df[complete_df["game_id"] == 'uXFQ2zgI-DUrgwlS']['phase_id'].unique()

array([0, 1, 2, 3, 4], dtype=object)

In [121]:
game_phase_df_list = []
for idx, game_id in enumerate(unique_games):
    print(idx, game_id)
    s_dict = {}
    d_dict = {}
    _id = 1
    unique_phases = complete_df[complete_df["game_id"] == game_id]['phase_id'].unique()
    for phase in unique_phases:
        # print("phase number is", phase)
        phase_df = complete_df[complete_df["phase_id"] == phase]
        res_df, s_dict, d_dict, _id = assign_unit_id(phase_df, s_dict, d_dict, _id)
        game_phase_df_list.append(phase_df)

0 uXFQ2zgI-DUrgwlS


ValueError: too many values to unpack (expected 2)

In [122]:
cdf = pd.concat(game_phase_df_list)

ValueError: No objects to concatenate

In [123]:
cdf

,game_id,phase_id,phase_name,coordinator,type,current_location,action,results,via,impact_location,unique_unit_id
0,uXFQ2zgI-DUrgwlS,0,S1901M,AUSTRIA,A,BUD,-,[],N,SER,1
1,uXFQ2zgI-DUrgwlS,0,S1901M,ENGLAND,F,LON,-,[],N,NTH,2
2,uXFQ2zgI-DUrgwlS,0,S1901M,FRANCE,A,MAR,-,[],N,SPA,3
3,uXFQ2zgI-DUrgwlS,0,S1901M,GERMANY,F,KIE,H,[],N,[],4
4,uXFQ2zgI-DUrgwlS,0,S1901M,ITALY,A,ROM,H,[],N,[],5
...,...,...,...,...,...,...,...,...,...,...,...
2512,7uKSGh-EG86tfpyh,31,S1908R,ENGLAND,N,N,N,N,N,N,-1
2513,7uKSGh-EG86tfpyh,31,S1908R,FRANCE,N,N,N,N,N,N,-1
2514,7uKSGh-EG86tfpyh,31,S1908R,GERMANY,N,N,N,N,N,N,-1
2515,7uKSGh-EG86tfpyh,31,S1908R,ITALY,N,N,N,N,N,N,-1


In [ ]:
print(cdf[cdf["game_id"] == "uXFQ2zgI-DUrgwlS"].to_string())

In [45]:
with open('out_df.txt', 'w') as f:
    print(cdf.groupby(["game_id","coordinator","phase_id"])["unique_unit_id"].nunique().to_string(), file=f)

In [ ]:
games

In [ ]:
complete_df[complete_df["game_id"] == 'Fdp3XM14WLTn5vBF']

In [ ]:
games_jsons